In [1]:
# Instalação das bibliotecas necessárias
!pip install pdfminer.six openai faiss-cpu pandas reportlab numpy

  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached pillow-12.0.0-cp310-cp310-win_amd64.whl.metadata (9.0 kB)
   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   --------- ------------------------------ 1.3/5.6 MB 8.4 MB/s eta 0:00:01
   ------------------ --------------------- 2.6/5.6 MB 7.9 MB/s eta 0:00:01
   -------------------------- ------------- 3.7/5.6 MB 6.4 MB/s eta 0:00:01
   ------------------------------- -------- 4.5/5.6 MB 5.7 MB/s eta 0:00:01
   ----------------------------------- ---- 5.0/5.6 MB 5.2 MB/s eta 0:00:01
   -------


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install python-dotenv


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import glob
import numpy as np
import pandas as pd
from typing import List, Dict, Tuple
from dotenv import load_dotenv

# Bibliotecas de PDF e Texto
from pdfminer.high_level import extract_text
from io import StringIO

# OpenAI e Vetores
from openai import OpenAI
import faiss

# Relatórios
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

# Load environment variables from .env file
load_dotenv()

# --- CONFIGURAÇÃO ---
# Substitua pela sua chave API ou defina na variável de ambiente
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
client = OpenAI()

DIRS = {
    "LATTES": "D:/PycharmProjects/PreparadorLattes/lattes/lattes.pdf",  # Caminho do arquivo Lattes
    "CERTIFICADOS": "D:/PycharmProjects/PreparadorLattes/certificados/" # Pasta com os PDFs
}

# Modelo de Embedding (mais barato e eficiente para busca)
EMBEDDING_MODEL = "text-embedding-3-small"
# Modelo de LLM para raciocínio
LLM_MODEL = "gpt-4o"

In [2]:
def extract_text_from_pdf(pdf_path: str) -> str:
    """Extrai texto bruto de um arquivo PDF."""
    print(f"Lendo o arquivo PDF: {pdf_path}")
    try:
        text = extract_text(pdf_path)
        if text.strip():
            print(
                f"Texto extraído com sucesso do arquivo: {pdf_path[:50]}...")  # Exibir os primeiros 50 caracteres do caminho
        else:
            print(f"Arquivo aparentemente vazio: {pdf_path}")
        return text
    except Exception as e:
        print(f"Erro ao ler {pdf_path}: {e}")
        return ""

def chunk_lattes_data(lattes_text: str) -> List[str]:
    """
    Quebra o texto do Lattes em itens verificáveis.
    Estratégia: Quebrar por linhas duplas ou blocos significativos.
    Filtra linhas muito curtas (cabeçalhos, números de página).
    """
    print("Iniciando o processo de chunking do Lattes...")
    raw_lines = [line.strip() for line in lattes_text.split('\n') if line.strip()]
    print(f"Total de linhas no texto Lattes: {len(raw_lines)}")

    items = []
    buffer = ""

    for line in raw_lines:
        if len(line) < 4:
            print(f"Linha ignorada (muito curta): '{line}'")
            continue
        buffer += " " + line
        if len(buffer) > 150:
            items.append(buffer.strip())
            print(f"Novo item detectado: {buffer[:50]}...")  # Exibir os 50 primeiros caracteres
            buffer = ""

    if buffer:
        items.append(buffer.strip())
        print(f"Adicionando último item detectado: {buffer[:50]}...")

    print(f"Total de itens após chunking: {len(items)}")
    return items

def load_certificates(folder_path: str) -> List[Dict]:
    """Lê todos os PDFs da pasta de certificados."""
    print(f"Procurando arquivos PDF na pasta: {folder_path}")
    certificates = []
    files = glob.glob(os.path.join(folder_path, "*.pdf"))
    print(f"Total de arquivos encontrados: {len(files)}")

    for idx, file_path in enumerate(files, start=1):
        filename = os.path.basename(file_path)
        print(f"Lendo ({idx}/{len(files)}): {filename}")

        content = extract_text_from_pdf(file_path)

        if len(content) < 50:
            print(f"Aviso: O arquivo '{filename}' parece vazio ou pode ser um PDF escaneado.")

        certificates.append({
            "filename": filename,
            "content": content,
            "path": file_path,
        })

    print(f"Total de certificados processados: {len(certificates)}")
    return certificates


In [3]:
def get_embedding(text: str, model=EMBEDDING_MODEL):
    """Gera o vetor numérico para um texto."""
    text = text.replace("\n", " ")
    # Truncate to fit limits if necessary, though simple texts are usually fine
    return client.embeddings.create(input=[text], model=model).data[0].embedding

def create_faiss_index(certificates: List[Dict]):
    """Cria o índice FAISS com os conteúdos dos certificados."""
    print("Gerando embeddings para os certificados...")
    embeddings = []
    valid_indices = []

    for idx, cert in enumerate(certificates):
        if not cert['content'].strip():
            continue

        emb = get_embedding(cert['content'])
        embeddings.append(emb)
        valid_indices.append(idx)

    if not embeddings:
        raise ValueError("Nenhum conteúdo de texto extraído dos certificados.")

    # Converte para numpy array float32 (exigido pelo FAISS)
    dataset = np.array(embeddings).astype("float32")

    # Dimensão do vetor (1536 para text-embedding-3-small)
    dimension = dataset.shape[1]

    # Cria índice L2 (Euclidiano)
    index = faiss.IndexFlatL2(dimension)
    index.add(dataset)

    return index, valid_indices

In [4]:
def verify_items(lattes_items: List[str], certificates: List[Dict], index, valid_cert_indices):
    results = []
    matched_certificates_files = set()

    print(f"Iniciando verificação de {len(lattes_items)} itens do Lattes...")

    for lattes_item in lattes_items:
        # 1. Busca Vetorial: Achar o candidato mais provável
        query_emb = np.array([get_embedding(lattes_item)]).astype("float32")
        k = 1 # Top 1 candidato
        distances, indices = index.search(query_emb, k)

        # Recupera o certificado sugerido pelo FAISS
        cert_idx_in_list = valid_cert_indices[indices[0][0]]
        candidate_cert = certificates[cert_idx_in_list]

        # 2. Validação Lógica com GPT-4 (LLM como Juiz)
        # O prompt é crucial aqui para evitar alucinações
        prompt = f"""
        Você é um auditor acadêmico rigoroso.

        ITEM DO CURRÍCULO LATTES:
        "{lattes_item}"

        CONTEÚDO DO CERTIFICADO (Arquivo: {candidate_cert['filename']}):
        "{candidate_cert['content'][:2000]}" ... (truncado)

        A pergunta: O conteúdo do certificado comprova EXATAMENTE ou PARCIALMENTE o item do currículo?
        Responda APENAS com um formato JSON: {{"match": true/false, "reason": "explicação curta"}}
        Se for uma correspondência fraca ou assunto diferente, responda false.
        """

        try:
            response = client.chat.completions.create(
                model=LLM_MODEL,
                messages=[{"role": "user", "content": prompt}],
                response_format={"type": "json_object"}
            )
            import json
            evaluation = json.loads(response.choices[0].message.content)

            is_match = evaluation.get('match', False)
            reason = evaluation.get('reason', 'Sem justificativa')

            status = "CONFORME" if is_match else "NÃO CONFORME (Certificado não encontrado ou incompatível)"
            matched_file = candidate_cert['filename'] if is_match else None

            if is_match:
                matched_certificates_files.add(candidate_cert['filename'])

            results.append({
                "Lattes_Item": lattes_item[:100] + "...", # Truncar para visualização
                "Status": status,
                "Certificado_Sugerido": candidate_cert['filename'],
                "LLM_Justificativa": reason
            })

        except Exception as e:
            print(f"Erro na validação LLM: {e}")

    return pd.DataFrame(results), matched_certificates_files

In [5]:
# 1. Executar o Pipeline
print("--- Lendo Lattes ---")
lattes_text = extract_text_from_pdf(DIRS["LATTES"])
lattes_items = chunk_lattes_data(lattes_text)

print("--- Lendo Certificados ---")
certs_data = load_certificates(DIRS["CERTIFICADOS"])

# Se não houver certificados ou lattes, parar
if not lattes_items or not certs_data:
    print("Erro: Faltam dados para processar.")
else:
    # 2. Indexação
    index, valid_indices = create_faiss_index(certs_data)

    # 3. Verificação (Lattes -> Certificados)
    df_results, matched_files = verify_items(lattes_items, certs_data, index, valid_indices)

    # 4. Verificação Inversa (Certificados -> Lattes)
    # Quais certificados estão na pasta mas não foram usados para validar nada?
    all_files = set([c['filename'] for c in certs_data])
    orphan_certs = all_files - matched_files

    # Exibir resultados na tela
    print("\n=== RESUMO DA ANÁLISE ===")
    print(df_results['Status'].value_counts())
    print(f"\nCertificados Sobrando (Órfãos): {len(orphan_certs)}")

    # 5. Geração do Relatório PDF
    def generate_pdf_report(dataframe, orphans, filename="Relatorio_Conformidade.pdf"):
        c = canvas.Canvas(filename, pagesize=letter)
        width, height = letter
        y = height - 40

        c.setFont("Helvetica-Bold", 16)
        c.drawString(30, y, "Relatório de Auditoria Lattes")
        y -= 30

        c.setFont("Helvetica", 10)
        c.drawString(30, y, "Itens analisados do Currículo Lattes e seus respectivos status:")
        y -= 20

        # Loop pelos itens (simplificado para o exemplo)
        for _, row in dataframe.iterrows():
            if y < 100: # Nova página
                c.showPage()
                y = height - 40
                c.setFont("Helvetica", 10)

            status_color = (0, 0.5, 0) if "CONFORME" in row['Status'] else (1, 0, 0) # Verde ou Vermelho

            c.setFillColorRGB(0, 0, 0)
            c.drawString(30, y, f"Item: {row['Lattes_Item']}")
            y -= 12

            c.setFillColorRGB(*status_color)
            c.drawString(30, y, f"Status: {row['Status']}")
            y -= 12

            c.setFillColorRGB(0.3, 0.3, 0.3)
            c.drawString(30, y, f"Justificativa: {row['LLM_Justificativa']}")
            y -= 25 # Espaço entre itens

        # Seção de Órfãos
        if y < 150: c.showPage(); y = height - 40

        y -= 20
        c.setFillColorRGB(0, 0, 0)
        c.setFont("Helvetica-Bold", 12)
        c.drawString(30, y, "Certificados presentes na pasta mas NÃO citados no Lattes:")
        y -= 20
        c.setFont("Helvetica", 10)

        for orphan in orphans:
            c.drawString(40, y, f"- {orphan}")
            y -= 15

        c.save()
        print(f"Relatório gerado com sucesso: {filename}")

    # Gerar
    generate_pdf_report(df_results, orphan_certs)

--- Lendo Lattes ---
Lendo o arquivo PDF: D:/PycharmProjects/PreparadorLattes/lattes/lattes.pdf
Texto extraído com sucesso do arquivo: D:/PycharmProjects/PreparadorLattes/lattes/lattes....
Iniciando o processo de chunking do Lattes...
Total de linhas no texto Lattes: 1111
Novo item detectado:  Paulo Victor dos Santos Endereço para acessar est...
Novo item detectado:  Sou um profissional com mais de 16 anos de experi...
Novo item detectado:  habilidades de comunicação, pensamento crítico e ...
Novo item detectado:  novos  desafios.  Meu  objetivo  é  continuar  ap...
Novo item detectado:  espiritual,  buscando  profundidade  em  minhas  ...
Novo item detectado:  mundo.Minha expertise inclui o desenvolvimento e ...
Novo item detectado:  Também tenho conhecimento em linguagens como SQL,...
Novo item detectado:  acadêmica  inclui  um  doutorado  em  Engenharia ...
Novo item detectado:  e  um  bacharelado  em  Análise  de  Sistemas  pe...
Novo item detectado:  Nome em citações bibliográfica

Cannot set gray non-stroke color because /'P1' is an invalid float value


Texto extraído com sucesso do arquivo: D:/PycharmProjects/PreparadorLattes/certificados\C...
Lendo (14/36): Coursera U8G96N4ZLHLK.pdf
Lendo o arquivo PDF: D:/PycharmProjects/PreparadorLattes/certificados\Coursera U8G96N4ZLHLK.pdf
Texto extraído com sucesso do arquivo: D:/PycharmProjects/PreparadorLattes/certificados\C...
Lendo (15/36): db2d0d559ffc4e758b7b2d07d9d3fc71.pdf
Lendo o arquivo PDF: D:/PycharmProjects/PreparadorLattes/certificados\db2d0d559ffc4e758b7b2d07d9d3fc71.pdf
Texto extraído com sucesso do arquivo: D:/PycharmProjects/PreparadorLattes/certificados\d...
Lendo (16/36): Declaração - Paulo Victor dos Santos - Aplicações Éticas de IA - Assinado.pdf
Lendo o arquivo PDF: D:/PycharmProjects/PreparadorLattes/certificados\Declaração - Paulo Victor dos Santos - Aplicações Éticas de IA - Assinado.pdf
Texto extraído com sucesso do arquivo: D:/PycharmProjects/PreparadorLattes/certificados\D...
Lendo (17/36): Declaração - Paulo Victor dos Santos - Automatizando Processos com IA - Assi

PermissionError: [Errno 13] Permission denied: 'Relatorio_Conformidade.pdf'